#### Quick Tour

<!-- !pip install transformers datasets
!pip install pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
 -->
##### 1. Pipeline 

The [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline) downloads and caches a default [pretrained model](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) and tokenizer for sentiment analysis. 
Now you can use the classifier on your target text:

In [1]:
import torch
from transformers import pipeline


def sentiment_analysis():
    classifier = pipeline("sentiment-analysis")
    print(classifier("We are very happy to show you the 🤗 Transformers library."))



2023-10-23 14:17:12.557099: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 14:17:12.557128: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 14:17:12.557143: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 14:17:12.561001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


If you have more than one input, pass your inputs as a list to the [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline) to return a list of dictionaries:

In [2]:
def multiple_input_classifier():
    results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
    for result in results:
        print(f"label: {result['label']}, with score: {round(result['score'], 4)}")


Load an audio dataset (see the 🤗 Datasets [Quick Start](https://huggingface.co/docs/datasets/quickstart#audio) for more details) you’d like to iterate over. For example, load the [MInDS-14](https://huggingface.co/datasets/PolyAI/minds14) dataset:

and You need to make sure the sampling rate of the dataset matches the sampling rate [facebook/wav2vec2-base-960h](https://huggingface.co/facebook/wav2vec2-base-960h) was trained on:

In [14]:

def speech_recognition():
    speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
    from datasets import load_dataset, Audio
    dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
    dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))
    # The audio files are automatically loaded and resampled when calling the "audio" column. 
    # Extract the raw waveform arrays from the first 4 samples and pass it as a list to the pipeline:
    result = speech_recognizer(dataset[:4]["audio"])
    print([d["text"] for d in result])
    
    # expected output:
    # ['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', 
    # "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", 
    # "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 
    # 'HOW DO I FURN A JOINA COUT']
 
speech_recognition()   

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']


For larger datasets where the inputs are big (like in speech or vision), you’ll want to pass a generator instead of a list to load all the inputs in memory. Take a look at the [pipeline API reference](https://huggingface.co/docs/transformers/main_classes/pipelines) for more information.


The **pipeline()** can accommodate any model from the [Hub](https://huggingface.co/models), making it easy to adapt the [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline) for other use-cases. For example, if you’d like a model capable of handling French text, use the tags on the Hub to filter for an appropriate model. The top filtered result returns a multilingual [BERT model](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) finetuned for sentiment analysis you can use for French text:

##### 2. Use another model and tokenizer in the pipeline
The [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline) can accommodate any model from the [Hub](https://huggingface.co/models), making it easy to adapt the [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline) for other use-cases. For example, if you’d like a model capable of handling French text, use the tags on the Hub to filter for an appropriate model. The top filtered result returns a multilingual [BERT model](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) finetuned for sentiment analysis you can use for French text:

In [4]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

Use [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/v4.34.0/en/model_doc/auto#transformers.AutoModelForSequenceClassification) and [AutoTokenizer](https://huggingface.co/docs/transformers/v4.34.0/en/model_doc/auto#transformers.AutoTokenizer) to load the pretrained model and it’s associated tokenizer (more on an AutoClass in the next section):

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Specify the model and tokenizer in the [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline), and now you can apply the **classifier** on French text:

In [6]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")
# epxected output: [{'label': '5 stars', 'score': 0.7273}]

[{'label': '5 stars', 'score': 0.7272652387619019}]

If you can’t find a model for your use-case, you’ll need to finetune a pretrained model on your data. Take a look at our [finetuning tutorial](https://huggingface.co/docs/transformers/training) to learn how. Finally, after you’ve finetuned your pretrained model, please consider [sharing](https://huggingface.co/docs/transformers/model_sharing) the model with the community on the Hub to democratize machine learning for everyone! 🤗

##### 3. AutoClass

[Video Tutorial - Instantiate a Transformer model pytorch](https://youtu.be/AhChOFRegn4)

Under the hood, the [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/v4.34.0/en/model_doc/auto#transformers.AutoModelForSequenceClassification) and [AutoTokenizer](https://huggingface.co/docs/transformers/v4.34.0/en/model_doc/auto#transformers.AutoTokenizer) classes work together to power the [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline) you used above. An [AutoClass](https://huggingface.co/docs/transformers/model_doc/auto) is a shortcut that automatically retrieves the architecture of a pretrained model from its name or path. You only need to select the appropriate AutoClass for your task and it’s associated preprocessing class.

Let’s return to the example from the previous section and see how you can use the AutoClass to replicate the results of the [pipeline()](https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/pipelines#transformers.pipeline).

The AutoConfig API allows you to **instantiate** the configuration of a pretrianed model from any checkpoint.

In [26]:
from transformers import AutoConfig
def autoconfig_from_pretained():
    bert_config = AutoConfig.from_pretrained("bert-base-cased")
    print(type(bert_config))
    
    gpt2_config = AutoConfig.from_pretrained("gpt2")
    print(type(gpt2_config))
    
    bart_config = AutoConfig.from_pretrained("facebook/bart-base")
    print(type(bart_config))



In [24]:
# same architecture as bert-base-cased

from transformers import BertConfig, BertModel

# Note that these model only have same configuration (e.g. input size, net structure, output size)
# but weights are randomly initialized if pretrained weights are not loaded
def initialize_bert_config_model():
    bert_config = BertConfig.from_pretrained("bert-base-cased")
    bert_model = BertModel(bert_config)

    # specify num_hidden_layers parameter if you want use 10 layers instead of 12 layers in bert
    bert_config = BertConfig.from_pretrained("bert-base-cased", num_hidden_layers=10)
    # here BertModel should be a 'callalbe' class
    bert_model = BertModel(bert_config)

# saving a model
def save_a_model():
    bert_config = BertConfig.from_pretrained("bert-base-cased")
    bert_model = BertModel(bert_config)
    # some training code
    bert_model.save_pretrained("my-bert-model")

# reloading a saved model
def reload_a_saved_model():
    bert_model = BertModel.from_pretrained("my-bert-model")
# this is what BertConfig look like
# probably in json format

    # BertConfig {
    #     "architectures": [
    #         "BertForMaskedLM"
    #     ],
    #     "attention_probs_dropout_prob": 0000,
    #     "gradient_checkpointing": 0000,
    #     "hidden_act": "gelu", 
    #     "hidden_dropout_prob": 0.1,
    #     "hidden_size": 768, 
    #     "initializer_range": 0.02, 
    #     "intermediate_size": 3072, 
    #     "layer_norm_eps": 1e-12, 
    #     "max_position_embeddings": 512, 
    #     "model_type": "bert",
    #     "num_attention_heads": 12, 
    #     "num_hidden_layers": 12, 
    #     "pad_token_id": 0, 
    #     "position_embedding_type": "absolute", 
    #     "transformers_version": "4.6.0.dev0",
    #     "type_vocab_size": 2, 
    #     "use_cache": true, 
    #     "vocab_size": 28996
    # }

##### 4. AutoTokenizer
A tokenizer is responsible for preprocessing text into an array of numbers as inputs to a model. There are multiple rules that govern the tokenization process, including how to split a word and at what level words should be split (learn more about tokenization in the [tokenizer summary](https://huggingface.co/docs/transformers/tokenizer_summary)). The most important thing to remember is you need to instantiate a tokenizer with the same model name to ensure you’re using the same tokenization rules a model was pretrained with.

Load a tokenizer with [AutoTokenizer](https://huggingface.co/docs/transformers/v4.34.0/en/model_doc/auto#transformers.AutoTokenizer):

In [15]:
from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Pass your text to the tokenizer:
encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encoding)

# expected output:
# {'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102],
#  'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


The tokenizer returns a dictionary containing:

 * [input_ids](https://huggingface.co/docs/transformers/glossary#input-ids): numerical representations of your tokens.
 * [attention_mask](https://huggingface.co/docs/transformers/.glossary#attention-mask): indicates which tokens should be attended to.

A tokenizer can also accept a list of inputs, and pad and truncate the text to return a batch with uniform length:

In [16]:
def define_pt_batch():
    pt_batch = tokenizer(
        ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

Check out the [preprocess](https://huggingface.co/docs/transformers/preprocessing) tutorial for more details about tokenization, and how to use an [AutoImageProcessor](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoImageProcessor), [AutoFeatureExtractor](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoFeatureExtractor) and [AutoProcessor](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoProcessor) to preprocess image, audio, and multimodal inputs.

##### 5. AutoModel

🤗 Transformers provides a simple and unified way to load pretrained instances. This means you can load an [AutoModel](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoModel) like you would load an [AutoTokenizer](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoTokenizer). The only difference is selecting the correct [AutoModel](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoModel) for the task. For text (or sequence) classification, you should load [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoModelForSequenceClassification):

See the [task summary](https://huggingface.co/docs/transformers/task_summary) for tasks supported by an [AutoModel](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoModel) class.

In [ ]:
def auto_model():
    from transformers import AutoModelForSequenceClassification
    
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
    
    # Now pass your preprocessed batch of inputs directly to the model. You just have to unpack the dictionary by adding **:
    
    pt_outputs = pt_model(**pt_batch)
    
    # The model outputs the final activations in the *logits* attribute. 
    # Apply the softmax function to the *logits* to retrieve the probabilities:
    
    from torch import nn
    
    pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
    print(pt_predictions)

All 🤗 Transformers models (PyTorch or TensorFlow) output the tensors before the final activation function (like softmax) because the final activation function is often fused with the loss. Model outputs are special dataclasses so their attributes are autocompleted in an IDE. The model outputs behave like a tuple or a dictionary (you can index with an integer, a slice or a string) in which case, attributes that are None are ignored.

##### 6. Save a model

Once your model is fine-tuned, you can save it with its tokenizer using [PreTrainedModel.save_pretrained()](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/model#transformers.PreTrainedModel.save_pretrained):

In [ ]:
def save_a_model():
    pt_save_directory = "./pt_save_pretrained"
    tokenizer.save_pretrained(pt_save_directory)
    pt_model.save_pretrained(pt_save_directory)

When you are ready to use the model again, reload it with [PreTrainedModel.from_pretrained()](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/model#transformers.PreTrainedModel.from_pretrained):

In [ ]:
pt_model = AutoModelForSequenceClassification.from_pretrained("./pt_save_pretrained")

One particularly cool 🤗 Transformers feature is the ability to save a model and reload it as either a PyTorch or TensorFlow model. The *from_pt* or *from_tf parameter* can convert the model from one framework to the other:

In [ ]:
from transformers import AutoModel

def convert_tf_to_pt():
    tokenizer = AutoTokenizer.from_pretrained(tf_save_directory)
    # the following line load a TensorFlow(tf) format model and convert to pytorch format model
    # by specifying parameter 'from_tf=True'
    pt_model = AutoModelForSequenceClassification.from_pretrained(tf_save_directory, from_tf=True)


##### 7. Custom model builds

You can modify the model’s configuration class to change how a model is built. The configuration specifies a model’s attributes, such as the number of hidden layers or attention heads. You start from scratch when you initialize a model from a custom configuration class. The model attributes are randomly initialized, and you’ll need to train the model before you can use it to get meaningful results.

Start by importing [AutoConfig](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoConfig), and then load the pretrained model you want to modify. Within [AutoConfig.from_pretrained()](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.AutoConfig.from_pretrained), you can specify the attribute you want to change, such as the number of attention heads:


In [ ]:
def auto_Config():
    from transformers import AutoConfig
    my_config = AutoConfig.from_pretrained("distilbert-base-uncased", n_heads=12)

You can also create a model from your custom configuration with [AutoModel.from_config()](https://huggingface.co/docs/transformers/v4.34.1/en/model_doc/auto#transformers.FlaxAutoModelForVision2Seq.from_config):

In [ ]:
# from transformers import AutoModel

# my_model = AutoModel.from_config(my_config)

#### Trainer - a PyTorch optimized training loop

All models are a standard [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) so you can use them in any typical training loop. While you can write your own training loop, 🤗 Transformers provides a [Trainer](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Trainer) class for PyTorch, which contains the basic training loop and adds additional functionality for features like distributed training, mixed precision, and more.

Depending on your task, you’ll typically pass the following parameters to [Trainer](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Trainer):

1. You’ll start with a [PreTrainedModel](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/model#transformers.PreTrainedModel) or a [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module):


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

2. [TrainingArguments](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.TrainingArguments) contains the model hyperparameters you can change like learning rate, batch size, and the number of epochs to train for. The default values are used if you don’t specify any training arguments:


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="path/to/save/folder/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
)

3. Load a preprocessing class like a tokenizer, image processor, feature extractor, or processor:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

4. Load a dataset:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")  # doctest: +IGNORE_RESULT

5. Create a function to tokenize the dataset:

In [ ]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Then apply it over the entire dataset with map:

In [ ]:
dataset = dataset.map(tokenize_dataset, batched=True)

6. A [DataCollatorWithPadding](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/data_collator#transformers.DataCollatorWithPadding) to create a batch of examples from your dataset:

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Now gather all these classes in [Trainer](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Trainer):

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

When you’re ready, call [train()](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Trainer.train) to start training:

In [ ]:
trainer.train()

### NOTE:

For tasks - like translation or summarization - that use a sequence-to-sequence model, use the [Seq2SeqTrainer](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Seq2SeqTrainer) and [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments) classes instead.

You can customize the training loop behavior by subclassing the methods inside [Trainer](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Trainer). This allows you to customize features such as the loss function, optimizer, and scheduler. Take a look at the [Trainer](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Trainer) reference for which methods can be subclassed.

The other way to customize the training loop is by using [Callbacks](https://huggingface.co/docs/transformers/main_classes/callbacks). You can use callbacks to integrate with other libraries and inspect the training loop to report on progress or stop the training early. Callbacks do not modify anything in the training loop itself. To customize something like the loss function, you need to subclass the [Trainer](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.Trainer) instead.